In [1]:
#Import modules
%matplotlib inline

#handling data
import pandas as pd
import numpy as np
from scipy import stats
from operator import itemgetter

#handling information
import re
import json

#handling plots
import matplotlib.pyplot as plt
import seaborn as sns

#for network creation
import networkx as nx
import tsm
import collections
import community
import copy
import csv
import networkx as nx
import operator
import random
import re

In [2]:
# Read json into a pandas dataframe
tweets_df = pd.read_csv("data/20220801-20220831-notverified_final")

In [3]:
tweets_df.columns

Index(['Unnamed: 0', 'id', 'created_at', 'user_id', 'text', 'author',
       'username'],
      dtype='object')

In [4]:
# dataを抽出
tweets_final = tweets_df[["username",'text','created_at']]

In [6]:
tweets_final
tweets_final.to_csv("data/processed/tweets_08010831_notverified_freelon.csv")

In [7]:
tweet_data = 'data/processed/tweets_08010831_notverified_freelon.csv'
# top_communities = tsm.get_top_communities(edgelist)
# ei_indices_A = tsm.calc_ei(top_communities_A.node_list,edgelist_A,'ON')
# ei_indices_B = tsm.calc_ei(top_communities_B.node_list,edgelist_B,'ON')
# top_rts_A = tsm.get_top_rts(tweet_file_A,top_communities_A.node_list)
# top_rts_B = tsm.get_top_rts(tweet_file_B,top_communities_B.node_list)
# community_matches = tsm.match_communities(top_communities_A.node_list,top_communities_B.node_list)
# intermediaries_A = tsm.get_intermediaries(top_communities_A.node_list,edgelist_A)
# intermediaries_B = tsm.get_intermediaries(top_communities_B.node_list,edgelist_B)
# top_hashtags_A = tsm.get_top_hashtags(tweet_file_A,top_communities_A.node_list)
# top_hashtags_B = tsm.get_top_hashtags(tweet_file_B,top_communities_B.node_list)
# top_links_A = tsm.get_top_links(tweet_file_A,top_communities_A.node_list)
# top_links_B = tsm.get_top_links(tweet_file_B,top_communities_B.node_list)
# grid_A = tsm.shared_ties_grid(ei_indices_A,calc_propor=True)
# grid_B = tsm.shared_ties_grid(ei_indices_B,calc_propor=True)

# print("Now feel free to inspect the variables...")

In [9]:
tweets_final

,username,text,created_at
0,yko1998,RT @NOSUKE0607: 参院選で壺団体に世話になったと報じられてますけど自民党の「壺...,2022-08-31 23:59:53+00:00
1,M1UM4,RT @DemandSiren: 自分は元ネトウヨだったけど、選択的夫婦別姓が世間で話題にな...,2022-08-31 23:59:40+00:00
2,miconmam,RT @KSakanako: 「共同親権も単独親権も選べた方がいい、夫婦別姓問題も一緒」って...,2022-08-31 23:59:36+00:00
3,katatedukhi,RT @NOSUKE0607: 参院選で壺団体に世話になったと報じられてますけど自民党の「壺...,2022-08-31 23:59:30+00:00
4,jcpchigasakishi,RT @suyama_hatsumi: 選択的夫婦別姓に反対してきた「国際勝共連合」は旧統一...,2022-08-31 23:59:29+00:00
...,...,...,...
151506,8c8g,RT @wanpakuten: #サンモニ\n松原耕二氏\n「ただ共通するのは、家父長制的な...,2022-08-01 00:01:47+00:00
151507,zK2zOCQMudQ9kDl,RT @ppsh41_1945: 自民党の杉田水脈議員は「保育所や学童保育は共産主義による洗...,2022-08-01 00:01:25+00:00
151508,CrdFtVl6RJqV8L6,RT @aiko33151709: ネット上では流布していたカルト統一教会と自民党との密接な...,2022-08-01 00:01:02+00:00
151509,nana77rey1,日本の戸籍は現存最古からずーっと【夫婦別姓（結婚改姓しない）】で運用されてきました。\n\n...,2022-08-01 00:00:34+00:00


In [12]:
f = open('data/processed/tweets_08010831_notverified_freelon.csv','r',encoding = 'utf-8',errors = 'replace')

In [13]:
csvreader = csv.reader(f)
# tweet_data = csv.reader(f)
# tweet_data

In [14]:
authors = []
tweets = []
for row in csvreader:
      authors.append(re.sub('[^A-Za-z0-9_]','',str(row[1]).lower().strip()))
      tweets.append(' ' + row[2].lower() + ' ')
authors
tweets

[' text ',
 ' rt @nosuke0607: 参院選で壺団体に世話になったと報じられてますけど自民党の「壺割り方針」の発表で即日の手のひら返し。すげーなこの人。でこれまで力説していた選択性夫婦別姓反対や「同性愛がかわいそう発言」とか今後どーするのですか？\n\n【速報】井上義行参院議員\u3000賛同会員… ',
 ' rt @demandsiren: 自分は元ネトウヨだったけど、選択的夫婦別姓が世間で話題になり始めた頃、「これは賛成だわ！うち姉妹しかいなくて苗字継ぐ人いないし！」ってなった途端「反日工作員」の烙印をかつての仲間（だと思ってた人達）から押されたのはすっごくショックな出来事として… ',
 ' rt @ksakanako: 「共同親権も単独親権も選べた方がいい、夫婦別姓問題も一緒」って、夫婦別姓と同じように、本人の同意がなければ共同が認められない制度が目指されているなら、こんなに頑張って反対しないのよ。 ',
 ' rt @nosuke0607: 参院選で壺団体に世話になったと報じられてますけど自民党の「壺割り方針」の発表で即日の手のひら返し。すげーなこの人。でこれまで力説していた選択性夫婦別姓反対や「同性愛がかわいそう発言」とか今後どーするのですか？\n\n【速報】井上義行参院議員\u3000賛同会員… ',
 ' rt @suyama_hatsumi: 選択的夫婦別姓に反対してきた「国際勝共連合」は旧統一協会と表裏一体。自民党と結びつき調査に影響を与えた疑念も。\n\n少なくとも「これまでの調査は選択的夫婦別姓賛成派に傾きすぎている」と設問内容に影響を与えた『一部の国会議員』の名前を明らかに… ',
 ' rt @ksakanako: 「共同親権も単独親権も選べた方がいい、夫婦別姓問題も一緒」って、夫婦別姓と同じように、本人の同意がなければ共同が認められない制度が目指されているなら、こんなに頑張って反対しないのよ。 ',
 ' rt @ksakanako: 「共同親権も単独親権も選べた方がいい、夫婦別姓問題も一緒」って、夫婦別姓と同じように、本人の同意がなければ共同が認められない制度が目指されているなら、こんなに頑張って反対しないのよ。 ',
 ' 選択的夫婦別姓について、「高品質」な反対論には出会ったことがないので何とも。 http

In [15]:
final = []

tweets = [t.split('@') for t in tweets] #splits each tweet along @s
for n,chunk in enumerate(tweets):
    ment_users = [t[:re.search('[^A-Za-z0-9_]',t).start()].lower().strip() for t in chunk if re.search('[^A-Za-z0-9_]',t) is not None]
    final.extend([[authors[n],name] for name in ment_users if len(name.strip()) > 0])


In [16]:
final

[['yko1998', 'nosuke0607'],
 ['m1um4', 'demandsiren'],
 ['miconmam', 'ksakanako'],
 ['katatedukhi', 'nosuke0607'],
 ['jcpchigasakishi', 'suyama_hatsumi'],
 ['limea', 'ksakanako'],
 ['miree808', 'ksakanako'],
 ['ilovecat222', 'ksakanako'],
 ['renkon33211147', 'tayutau_ing'],
 ['renkon33211147', 'emil418'],
 ['nana77rey1', 'alloemloveu'],
 ['awozora7', 'yamtom'],
 ['aichirodojcp', 'suyama_hatsumi'],
 ['si6tabi', 'junmishina'],
 ['la_vie_en17rose', 'cdp_aichi10'],
 ['nothing_001', 'grapefruitsjui2'],
 ['nothing_001', 'grapefruitsjui2'],
 ['kxxk_kxxk_', 'nosuke0607'],
 ['r4w_j', 'kamiwaki'],
 ['agmatine12', 'runforther'],
 ['usagi_21_ballet', 'nosuke0607'],
 ['ichinott3', 'yamtom'],
 ['sunnysanio', 'ge5c6olpmakcie1'],
 ['sunnysanio', 'mt3678mt'],
 ['_kori003', 'kato76446907'],
 ['kenseikyuu', 'yamtom'],
 ['life_tweet_ing', 'ksakanako'],
 ['jcjmmlgiyjxyq1c', 'yamatonara999g1'],
 ['jcjmmlgiyjxyq1c', 'jcjmmlgiyjxyq1c'],
 ['3215san', 'yamtom'],
 ['psuzuki6', 'nosuke0607'],
 ['ra0k92yd', 'yamto

In [17]:
def save_csv(filename,data,use_quotes=False,double_to_single=True,file_mode='w',enc='utf-8',verbose=True): #this assumes a list of lists wherein the second-level list items contain no commas
    with open(filename,file_mode,encoding = enc) as out:
        for line in data:
            if type(line) is not list and type(line) is not tuple:
                line = [line] #forces all items in 'data' to be lists
            if use_quotes == True:
                if double_to_single == True:
                    row = '"' + '","'.join([str(i).replace('"',"'") for i in line]) + '"' + "\n"
                else:
                    row = '"' + '","'.join([str(i) for i in line]) + '"' + "\n"
            else:
                row = ','.join([str(i) for i in line]) + "\n"
            out.write(row)
    if verbose == True:
        print('Data saved to file "' + filename + '".')

In [18]:
def load_data(data,enc='utf-8',translate_unicode=False):
    if type(data) is str:
        csv_data = []
        with open(data,'r',encoding = enc,errors = 'replace') as f:
            if translate_unicode == True:
                reader = csv.reader((line.encode().decode('unicode_escape').replace('\0','') for line in f)) #remove NULL bytes
            else:
                reader = csv.reader((line.replace('\0','') for line in f)) #remove NULL bytes
            for row in reader:
                if row != []:
                    csv_data.append(row)
        print('Data loaded from file "' + data + '".')
        return csv_data
    else:
        print('Data loaded.')
        return copy.deepcopy(data)

In [17]:
outfile = '_edgelist.csv'
save_csv(outfile,final)

Data saved to file "_edgelist.csv".


In [18]:
top_communities = tsm.get_top_communities(final)

Data loaded.
Non-directed network created.
Community partition complete.
Total n of communities: 359
Modularity: 0.77
Community analysis complete. The top 10 communities in this network account for 68.87 % of all nodes.
And 66.78 % of all edges.


In [19]:
nodes_data = top_communities.node_list
nodes_data

[['oriak9', '1', '1512'],
 ['hotaka_tsukada', '6', '1393'],
 ['nipponkairagi', '11', '1344'],
 ['seiryukai', '21', '1218'],
 ['aiko33151709', '14', '1189'],
 ['quitamarco', '0', '585'],
 ['nana77rey1', '4', '322'],
 ['ssimtok', '16', '298'],
 ['yampisun40', '17', '197'],
 ['chizurua1', '25', '178'],
 ['shop_kakiko', '11', '124'],
 ['kenkenrokujapan', '17', '106'],
 ['mi89s1', '25', '104'],
 ['hosakanobuto', '16', '95'],
 ['5sasha5sasha5', '6', '92'],
 ['nwfle6q9vqtxb4q', '17', '71'],
 ['quotationmark95', '25', '64'],
 ['yoboudr_op', '4', '59'],
 ['id_seiron', '17', '55'],
 ['grayengineer', '17', '53'],
 ['sollevante2022', '4', '50'],
 ['w4f75x7mj9i2gdb', '17', '45'],
 ['oosakahaoosaka', '25', '44'],
 ['h_koyuyu', '17', '43'],
 ['yuki1988jp', '25', '41'],
 ['kakitama', '16', '41'],
 ['haguruma0830', '17', '38'],
 ['starship_glory', '21', '35'],
 ['powertopeople5', '25', '34'],
 ['run_and_walk_', '4', '32'],
 ['nikone_25', '21', '32'],
 ['jinn11', '25', '31'],
 ['pokhara1995', '4', '28']

In [20]:
top_communities.n_communities

359

In [21]:
top_communities.modularity

0.77

In [22]:
nodes_data

[['oriak9', '1', '1512'],
 ['hotaka_tsukada', '6', '1393'],
 ['nipponkairagi', '11', '1344'],
 ['seiryukai', '21', '1218'],
 ['aiko33151709', '14', '1189'],
 ['quitamarco', '0', '585'],
 ['nana77rey1', '4', '322'],
 ['ssimtok', '16', '298'],
 ['yampisun40', '17', '197'],
 ['chizurua1', '25', '178'],
 ['shop_kakiko', '11', '124'],
 ['kenkenrokujapan', '17', '106'],
 ['mi89s1', '25', '104'],
 ['hosakanobuto', '16', '95'],
 ['5sasha5sasha5', '6', '92'],
 ['nwfle6q9vqtxb4q', '17', '71'],
 ['quotationmark95', '25', '64'],
 ['yoboudr_op', '4', '59'],
 ['id_seiron', '17', '55'],
 ['grayengineer', '17', '53'],
 ['sollevante2022', '4', '50'],
 ['w4f75x7mj9i2gdb', '17', '45'],
 ['oosakahaoosaka', '25', '44'],
 ['h_koyuyu', '17', '43'],
 ['yuki1988jp', '25', '41'],
 ['kakitama', '16', '41'],
 ['haguruma0830', '17', '38'],
 ['starship_glory', '21', '35'],
 ['powertopeople5', '25', '34'],
 ['run_and_walk_', '4', '32'],
 ['nikone_25', '21', '32'],
 ['jinn11', '25', '31'],
 ['pokhara1995', '4', '28']

In [24]:
class eiObject:
    '''an object class with attributes for various EI-index-related data and metadata'''
    def __init__(self,n_nodes=None,ei_indices=None,internal_ties=None,external_ties=None,mean_ei=None,total_ties=None,received_ties=None,sent_ties=None,r_s=None,adj_in=None,adj_out=None):
        self.n_nodes = n_nodes
        self.ei_indices = ei_indices
        self.internal_ties = internal_ties
        self.external_ties = external_ties
        self.mean_ei = mean_ei
        self.total_ties = total_ties
        self.received_ties = received_ties
        self.sent_ties = sent_ties
        self.r_s = r_s
        self.adj_out = adj_out
        self.adj_in = adj_in

In [25]:

def _get_shared_ties(top_community_ids,top_edges,ei_int,ei_ext,n_nodes,verbose):
    adj_out = {} #sent ties point away from the focal community
    adj_in = {} #received ties point toward the focal community

    for i in top_community_ids:
        adj_out[i] = {}
        adj_in[i] = {}
        for j in top_edges:
            if j[2] == i and j[3] != i:
                if j[3] in adj_out[i]:
                    adj_out[i][j[3]] += 1
                else:
                    adj_out[i][j[3]] = 1
            if j[2] != i and j[3] == i:
                if j[2] in adj_in[i]:
                    adj_in[i][j[2]] += 1
                else:
                    adj_in[i][j[2]] = 1

    total_dict = {}
    received_dict = {}
    sent_dict = {}
    r_s_dict = {}

    for i in top_community_ids:
        total = ei_int[i]+ei_ext[i]
        total_dict[i] = total
        incoming = sum(adj_in[i].values())
        received_dict[i] = incoming
        outgoing = sum(adj_out[i].values())
        sent_dict[i] = outgoing
        r_s = round(incoming - outgoing,3)
        r_s_dict[i] = r_s

        if verbose == True:
            print("\nCommunity\tSize\tInternal\tReceived\tSent\tReceived - Sent")
            print(str(i)+"\t"+str(n_nodes[i])+"\t"+str(round(ei_int[i]/total,3))+"\t"+str(round(incoming/total,3))+"\t"+str(round(outgoing/total,3))+"\t"+str(round(r_s/total,3))+"\n")
            for j in top_community_ids:
                if i != j:
                    if j in adj_out[i]:
                        print(str(j)+"-s\t"+str(round(adj_out[i][j]/total,3)))
                    if j in adj_in[i]:
                        print(str(j)+"-r\t"+str(round(adj_in[i][j]/total,3)))

    prox_out = eiObject()
    prox_out.adj_in = collections.OrderedDict(sorted(adj_in.items()))
    prox_out.adj_out = collections.OrderedDict(sorted(adj_out.items()))
    prox_out.total_ties = collections.OrderedDict(sorted(total_dict.items()))
    prox_out.received_ties = collections.OrderedDict(sorted(received_dict.items()))
    prox_out.sent_ties = collections.OrderedDict(sorted(sent_dict.items()))
    prox_out.r_s = collections.OrderedDict(sorted(r_s_dict.items()))

    return prox_out

In [26]:
def calc_ei(nodes_data,edges_data,all_output=True,pause=False,weight_edges=True,verbose=True,save_prefix=''):
    if weight_edges == False:
        print("Calculating EI indices using *UNweighted* edges.\n")
    else:
        print("Calculating EI indices using *weighted* edges.\n")

    nodes = load_data(nodes_data)
    if nodes[0][0] == 'name':
        del nodes[0] #remove headers from CSV

    edges = load_data(edges_data)
    modclass = [i[1] for i in nodes]
    moduniq = {}

    for i in modclass: #get and count unique community IDs
        if i in moduniq:
            moduniq[i] += 1
        else:
            moduniq[i] = 1

    mu_top = sorted(moduniq,key=moduniq.get,reverse=True) #get all communities from node file
    top_nodes_list = [i for i in nodes if i[1] in mu_top]
    top_nodes = {node[0]:node[1] for node in top_nodes_list} #create dict of screen names and community IDs

    if weight_edges == False:
        edges = list(set([i[0] + "," + i[1] for i in edges])) #unweight the edgelist--multiple links to B from A count as one edge
        edges = [i.split(",") for i in edges]

    for edge in edges:
        if edge[0] in top_nodes:
            edge.append(top_nodes[edge[0]])

    for edge in edges:
        if edge[1] in top_nodes:
            edge.append(top_nodes[edge[1]])

    top_edges = [i for i in edges if len(i) == 4]

    ei_int = {}
    ei_ext = {}

    for i in mu_top:
        ei_int[i] = 0
        ei_ext[i] = 0
        for j in top_edges:
            if j[2] == i and j[3] == i:
                ei_int[i] += 1
            elif j[2] == i or j[3] == i:
                ei_ext[i] += 1

    ei_indices = {}

    for i in ei_int:
        ei_indices[i] = round((ei_ext[i]-ei_int[i])/(ei_ext[i]+ei_int[i]),3)

    ei_ord = collections.OrderedDict(sorted(ei_indices.items()))
    ei_int = collections.OrderedDict(sorted(ei_int.items()))
    ei_ext = collections.OrderedDict(sorted(ei_ext.items()))
    mean_ei = round(sum(ei_indices.values())/len(ei_indices.values()),3)

    if verbose == True:
        print("***EI indices***\n")
        print("Community\tEI index")
        for i in ei_ord:
            print(str(i)+"\t"+str(ei_ord[i]))

        print("Mean EI:\t",mean_ei)

    n_nodes = {}
    for i in nodes:
        if i[1] in n_nodes:
            n_nodes[i[1]] += 1
        else:
            n_nodes[i[1]] = 1

    if pause == True:
        input('Press any key to continue...')

    if all_output == True:
        ei_out = _get_shared_ties(mu_top,top_edges,ei_int,ei_ext,n_nodes,verbose)
    else:
        ei_out = eiObject()

    if len(save_prefix) > 0:
        ei_list = []
        for i in ei_ord:
            ei_list.append([str(i),str(ei_ord[i])])
        ei_list.insert(0,['Community IDs','EI indices'])
        save_csv(save_prefix + '_ei_indices.csv',ei_list)
    print("\n")

    ei_out.n_nodes = collections.OrderedDict(sorted(n_nodes.items()))
    ei_out.ei_indices = ei_ord
    ei_out.internal_ties = ei_int
    ei_out.external_ties = ei_ext
    ei_out.mean_ei = mean_ei
    return ei_out

In [27]:
calc_ei(nodes_data, final)

Calculating EI indices using *weighted* edges.

Data loaded.
Data loaded.
***EI indices***

Community	EI index
0	-0.476
1	-0.675
11	-0.636
14	-0.653
16	-0.299
17	-0.521
21	-0.602
25	-0.562
4	-0.449
6	-0.518
Mean EI:	 -0.539

Community	Size	Internal	Received	Sent	Received - Sent
1	1303	0.837	0.16	0.003	0.157

6-s	0.001
6-r	0.036
14-r	0.01
17-r	0.022
4-s	0.001
4-r	0.052
25-r	0.009
0-s	0.001
0-r	0.013
16-r	0.018

Community	Size	Internal	Received	Sent	Received - Sent
6	1280	0.759	0.164	0.077	0.087

1-s	0.031
1-r	0.001
21-r	0.001
14-s	0.01
14-r	0.042
17-r	0.001
4-s	0.008
4-r	0.033
25-s	0.003
25-r	0.025
0-s	0.014
0-r	0.024
16-s	0.012
16-r	0.037

Community	Size	Internal	Received	Sent	Received - Sent
11	1264	0.818	0.14	0.042	0.099

21-s	0.037
21-r	0.093
14-s	0.001
17-s	0.004
17-r	0.045
4-r	0.002
25-s	0.001

Community	Size	Internal	Received	Sent	Received - Sent
21	1211	0.801	0.089	0.11	-0.022

6-s	0.001
11-s	0.085
11-r	0.034
17-s	0.022
17-r	0.054
4-s	0.001
4-r	0.001
25-r	0.001
16-s	0.001

Commu

In [28]:
#  self.n_nodes = n_nodes
#         self.ei_indices = ei_indices
#         self.internal_ties = internal_ties
#         self.external_ties = external_ties
#         self.mean_ei = mean_ei
#         self.total_ties = total_ties
#         self.received_ties = received_ties
#         self.sent_ties = sent_ties
#         self.r_s = r_s
#         self.adj_out = adj_out
#         self.adj_in = adj_in

In [29]:
a = calc_ei(nodes_data, final)

Calculating EI indices using *weighted* edges.

Data loaded.
Data loaded.
***EI indices***

Community	EI index
0	-0.476
1	-0.675
11	-0.636
14	-0.653
16	-0.299
17	-0.521
21	-0.602
25	-0.562
4	-0.449
6	-0.518
Mean EI:	 -0.539

Community	Size	Internal	Received	Sent	Received - Sent
1	1303	0.837	0.16	0.003	0.157

6-s	0.001
6-r	0.036
14-r	0.01
17-r	0.022
4-s	0.001
4-r	0.052
25-r	0.009
0-s	0.001
0-r	0.013
16-r	0.018

Community	Size	Internal	Received	Sent	Received - Sent
6	1280	0.759	0.164	0.077	0.087

1-s	0.031
1-r	0.001
21-r	0.001
14-s	0.01
14-r	0.042
17-r	0.001
4-s	0.008
4-r	0.033
25-s	0.003
25-r	0.025
0-s	0.014
0-r	0.024
16-s	0.012
16-r	0.037

Community	Size	Internal	Received	Sent	Received - Sent
11	1264	0.818	0.14	0.042	0.099

21-s	0.037
21-r	0.093
14-s	0.001
17-s	0.004
17-r	0.045
4-r	0.002
25-s	0.001

Community	Size	Internal	Received	Sent	Received - Sent
21	1211	0.801	0.089	0.11	-0.022

6-s	0.001
11-s	0.085
11-r	0.034
17-s	0.022
17-r	0.054
4-s	0.001
4-r	0.001
25-r	0.001
16-s	0.001

Commu

## Dailydata

In [20]:
# subset data
tweets_data_20220801 = tweets_final[(tweets_final['created_at'] >= '2022-08-01') & (tweets_final['created_at'] < '2022-08-02')]
tweets_data_20220802 = tweets_final[(tweets_final['created_at'] >= '2022-08-02') & (tweets_final['created_at'] < '2022-08-03')]
tweets_data_20220803 = tweets_final[(tweets_final['created_at'] >= '2022-08-03') & (tweets_final['created_at'] < '2022-08-04')]
tweets_data_20220804 = tweets_final[(tweets_final['created_at'] >= '2022-08-04') & (tweets_final['created_at'] < '2022-08-05')]
tweets_data_20220805 = tweets_final[(tweets_final['created_at'] >= '2022-08-05') & (tweets_final['created_at'] < '2022-08-06')]
tweets_data_20220806 = tweets_final[(tweets_final['created_at'] >= '2022-08-06') & (tweets_final['created_at'] < '2022-08-07')]
tweets_data_20220807 = tweets_final[(tweets_final['created_at'] >= '2022-08-07') & (tweets_final['created_at'] < '2022-08-08')]
tweets_data_20220808 = tweets_final[(tweets_final['created_at'] >= '2022-08-08') & (tweets_final['created_at'] < '2022-08-09')]
tweets_data_20220809 = tweets_final[(tweets_final['created_at'] >= '2022-08-09') & (tweets_final['created_at'] < '2022-08-10')]
tweets_data_20220810 = tweets_final[(tweets_final['created_at'] >= '2022-08-10') & (tweets_final['created_at'] < '2022-08-11')]

tweets_data_20220811 = tweets_final[(tweets_final['created_at'] >= '2022-08-11') & (tweets_final['created_at'] < '2022-08-12')]
tweets_data_20220812 = tweets_final[(tweets_final['created_at'] >= '2022-08-12') & (tweets_final['created_at'] < '2022-08-13')]
tweets_data_20220813 = tweets_final[(tweets_final['created_at'] >= '2022-08-13') & (tweets_final['created_at'] < '2022-08-14')]
tweets_data_20220814 = tweets_final[(tweets_final['created_at'] >= '2022-08-14') & (tweets_final['created_at'] < '2022-08-15')]
tweets_data_20220815 = tweets_final[(tweets_final['created_at'] >= '2022-08-15') & (tweets_final['created_at'] < '2022-08-16')]
tweets_data_20220816 = tweets_final[(tweets_final['created_at'] >= '2022-08-16') & (tweets_final['created_at'] < '2022-08-17')]
tweets_data_20220817 = tweets_final[(tweets_final['created_at'] >= '2022-08-17') & (tweets_final['created_at'] < '2022-08-18')]
tweets_data_20220818 = tweets_final[(tweets_final['created_at'] >= '2022-08-18') & (tweets_final['created_at'] < '2022-08-19')]
tweets_data_20220819 = tweets_final[(tweets_final['created_at'] >= '2022-08-19') & (tweets_final['created_at'] < '2022-08-20')]
tweets_data_20220820 = tweets_final[(tweets_final['created_at'] >= '2022-08-20') & (tweets_final['created_at'] < '2022-08-21')]

tweets_data_20220821 = tweets_final[(tweets_final['created_at'] >= '2022-08-21') & (tweets_final['created_at'] < '2022-08-22')]
tweets_data_20220822 = tweets_final[(tweets_final['created_at'] >= '2022-08-22') & (tweets_final['created_at'] < '2022-08-23')]
tweets_data_20220823 = tweets_final[(tweets_final['created_at'] >= '2022-08-23') & (tweets_final['created_at'] < '2022-08-24')]
tweets_data_20220824 = tweets_final[(tweets_final['created_at'] >= '2022-08-24') & (tweets_final['created_at'] < '2022-08-25')]
tweets_data_20220825 = tweets_final[(tweets_final['created_at'] >= '2022-08-25') & (tweets_final['created_at'] < '2022-08-26')]
tweets_data_20220826 = tweets_final[(tweets_final['created_at'] >= '2022-08-26') & (tweets_final['created_at'] < '2022-08-27')]
tweets_data_20220827 = tweets_final[(tweets_final['created_at'] >= '2022-08-27') & (tweets_final['created_at'] < '2022-08-28')]
tweets_data_20220828 = tweets_final[(tweets_final['created_at'] >= '2022-08-28') & (tweets_final['created_at'] < '2022-08-29')]
tweets_data_20220829 = tweets_final[(tweets_final['created_at'] >= '2022-08-29') & (tweets_final['created_at'] < '2022-08-30')]
tweets_data_20220830 = tweets_final[(tweets_final['created_at'] >= '2022-08-30') & (tweets_final['created_at'] < '2022-08-31')]
tweets_data_20220831 = tweets_final[(tweets_final['created_at'] >= '2022-08-31') & (tweets_final['created_at'] < '2022-09-01')]



In [21]:
#csv
tweets_data_20220801.to_csv("data/processed/tweets_20220801_notverified_freelon.csv")
tweets_data_20220802.to_csv("data/processed/tweets_20220802_notverified_freelon.csv")
tweets_data_20220803.to_csv("data/processed/tweets_20220803_notverified_freelon.csv")
tweets_data_20220804.to_csv("data/processed/tweets_20220804_notverified_freelon.csv")
tweets_data_20220805.to_csv("data/processed/tweets_20220805_notverified_freelon.csv")
tweets_data_20220806.to_csv("data/processed/tweets_20220806_notverified_freelon.csv")
tweets_data_20220807.to_csv("data/processed/tweets_20220807_notverified_freelon.csv")
tweets_data_20220808.to_csv("data/processed/tweets_20220808_notverified_freelon.csv")
tweets_data_20220809.to_csv("data/processed/tweets_20220809_notverified_freelon.csv")
tweets_data_20220810.to_csv("data/processed/tweets_20220810_notverified_freelon.csv")
tweets_data_20220811.to_csv("data/processed/tweets_20220811_notverified_freelon.csv")
tweets_data_20220812.to_csv("data/processed/tweets_20220812_notverified_freelon.csv")
tweets_data_20220813.to_csv("data/processed/tweets_20220813_notverified_freelon.csv")
tweets_data_20220814.to_csv("data/processed/tweets_20220814_notverified_freelon.csv")
tweets_data_20220815.to_csv("data/processed/tweets_20220815_notverified_freelon.csv")
tweets_data_20220816.to_csv("data/processed/tweets_20220816_notverified_freelon.csv")
tweets_data_20220817.to_csv("data/processed/tweets_20220817_notverified_freelon.csv")
tweets_data_20220818.to_csv("data/processed/tweets_20220818_notverified_freelon.csv")
tweets_data_20220819.to_csv("data/processed/tweets_20220819_notverified_freelon.csv")
tweets_data_20220820.to_csv("data/processed/tweets_20220820_notverified_freelon.csv")

tweets_data_20220821.to_csv("data/processed/tweets_20220821_notverified_freelon.csv")
tweets_data_20220822.to_csv("data/processed/tweets_20220822_notverified_freelon.csv")
tweets_data_20220823.to_csv("data/processed/tweets_20220823_notverified_freelon.csv")
tweets_data_20220824.to_csv("data/processed/tweets_20220824_notverified_freelon.csv")
tweets_data_20220825.to_csv("data/processed/tweets_20220825_notverified_freelon.csv")
tweets_data_20220826.to_csv("data/processed/tweets_20220826_notverified_freelon.csv")
tweets_data_20220827.to_csv("data/processed/tweets_20220827_notverified_freelon.csv")
tweets_data_20220828.to_csv("data/processed/tweets_20220828_notverified_freelon.csv")
tweets_data_20220829.to_csv("data/processed/tweets_20220829_notverified_freelon.csv")
tweets_data_20220830.to_csv("data/processed/tweets_20220830_notverified_freelon.csv")
tweets_data_20220831.to_csv("data/processed/tweets_20220831_notverified_freelon.csv")

In [60]:
#20220803

f = open('data/processed/tweets_20220831_notverified_freelon.csv','r',encoding = 'utf-8',errors = 'replace')
csvreader = csv.reader(f)
authors = []
tweets = []
for row in csvreader:
    authors.append(re.sub('[^A-Za-z0-9_]','',str(row[1]).lower().strip()))
    tweets.append(' ' + row[2].lower() + ' ')

final = []
tweets = [t.split('@') for t in tweets] #splits each tweet along @s
for n,chunk in enumerate(tweets):
    ment_users = [t[:re.search('[^A-Za-z0-9_]',t).start()].lower().strip() for t in chunk if re.search('[^A-Za-z0-9_]',t) is not None]
    final.extend([[authors[n],name] for name in ment_users if len(name.strip()) > 0])
outfile = '_edgelist_.csv'
save_csv(outfile,final)
top_communities = tsm.get_top_communities(final)
nodes_data = top_communities.node_list
calc_ei(nodes_data, final)

Data saved to file "_edgelist_.csv".
Data loaded.
Non-directed network created.
Community partition complete.
Total n of communities: 172
Modularity: 0.83
Community analysis complete. The top 10 communities in this network account for 74.01 % of all nodes.
And 73.68 % of all edges.
Calculating EI indices using *weighted* edges.

Data loaded.
Data loaded.
***EI indices***

Community	EI index
1	-0.702
11	-0.785
13	-0.486
17	-0.977
2	-0.746
25	-0.962
5	-0.681
7	-0.555
8	-0.937
9	-0.659
Mean EI:	 -0.749

Community	Size	Internal	Received	Sent	Received - Sent
1	634	0.851	0.148	0.001	0.146

5-r	0.034
11-r	0.03
7-r	0.024
9-r	0.017
8-r	0.001
2-r	0.021
13-s	0.001
13-r	0.02

Community	Size	Internal	Received	Sent	Received - Sent
5	601	0.84	0.125	0.035	0.09

1-s	0.035
11-r	0.02
7-r	0.059
9-r	0.012
2-r	0.009
13-r	0.024

Community	Size	Internal	Received	Sent	Received - Sent
11	555	0.893	0.047	0.06	-0.013

1-s	0.036
5-s	0.024
7-r	0.022
9-r	0.005
8-r	0.006
2-r	0.005
17-r	0.002
13-r	0.008

Community	Siz

#20220803

f = open('data/processed/tweets_20220803_notverified_freelon.csv','r',encoding = 'utf-8',errors = 'replace')
csvreader_20220803 = csv.reader(f)
authors_20220803 = []
tweets_20220803 = []
for row in csvreader_20220803:
    authors_20220803.append(re.sub('[^A-Za-z0-9_]','',str(row[1]).lower().strip()))
    tweets_20220803.append(' ' + row[2].lower() + ' ')

final_20220803 = []
tweets_20220803 = [t.split('@') for t in tweets_20220803] #splits each tweet along @s
for n,chunk in enumerate(tweets_20220803):
    ment_users = [t[:re.search('[^A-Za-z0-9_]',t).start()].lower().strip() for t in chunk if re.search('[^A-Za-z0-9_]',t) is not None]
    final_20220803.extend([[authors_20220803[n],name] for name in ment_users if len(name.strip()) > 0])
outfile = '_edgelist_20220803.csv'
save_csv(outfile,final_20220803)
top_communities_20220803 = tsm.get_top_communities(final_20220803)
nodes_data_20220803 = top_communities_20220803.node_list
calc_ei(nodes_data_20220803, final_20220803)

In [35]:
#20220804

f = open('data/processed/tweets_20220804_notverified_freelon.csv','r',encoding = 'utf-8',errors = 'replace')
csvreader_20220804 = csv.reader(f)
authors_20220804 = []
tweets_20220804 = []
for row in csvreader_20220804:
    authors_20220804.append(re.sub('[^A-Za-z0-9_]','',str(row[1]).lower().strip()))
    tweets_20220804.append(' ' + row[2].lower() + ' ')

final_20220804 = []
tweets_20220804 = [t.split('@') for t in tweets_20220804] #splits each tweet along @s
for n,chunk in enumerate(tweets_20220804):
    ment_users = [t[:re.search('[^A-Za-z0-9_]',t).start()].lower().strip() for t in chunk if re.search('[^A-Za-z0-9_]',t) is not None]
    final_20220804.extend([[authors_20220804[n],name] for name in ment_users if len(name.strip()) > 0])
outfile = '_edgelist_20220804.csv'
save_csv(outfile,final_20220804)
top_communities_20220804 = tsm.get_top_communities(final_20220804)
nodes_data_20220804 = top_communities_20220804.node_list
calc_ei(nodes_data_20220804, final_20220804)

Data saved to file "_edgelist_20220804.csv".
Data loaded.
Non-directed network created.
Community partition complete.
Total n of communities: 161
Modularity: 0.81
Community analysis complete. The top 10 communities in this network account for 80.0 % of all nodes.
And 79.83 % of all edges.
Calculating EI indices using *weighted* edges.

Data loaded.
Data loaded.
***EI indices***

Community	EI index
1	-0.987
10	-0.843
11	-0.622
12	-0.759
19	-0.607
2	-0.778
3	-0.588
30	-0.6
4	-0.676
8	-0.685
Mean EI:	 -0.714

Community	Size	Internal	Received	Sent	Received - Sent
1	1252	0.994	0.006	0.0	0.006

10-r	0.006
11-r	0.001

Community	Size	Internal	Received	Sent	Received - Sent
12	607	0.88	0.119	0.001	0.118

2-r	0.02
4-r	0.029
10-r	0.003
8-r	0.013
3-r	0.017
11-r	0.015
30-s	0.001
30-r	0.013
19-r	0.009

Community	Size	Internal	Received	Sent	Received - Sent
2	357	0.889	0.077	0.035	0.042

12-s	0.035
4-r	0.022
8-r	0.02
3-r	0.017
11-r	0.007
30-r	0.002
19-r	0.007

Community	Size	Internal	Received	Sent	Rece

In [36]:
#20220805

f = open('data/processed/tweets_20220805_notverified_freelon.csv','r',encoding = 'utf-8',errors = 'replace')
csvreader_20220805 = csv.reader(f)
authors_20220805 = []
tweets_20220805 = []
for row in csvreader_20220805:
    authors_20220805.append(re.sub('[^A-Za-z0-9_]','',str(row[1]).lower().strip()))
    tweets_20220805.append(' ' + row[2].lower() + ' ')

final_20220805 = []
tweets_20220805 = [t.split('@') for t in tweets_20220805] #splits each tweet along @s
for n,chunk in enumerate(tweets_20220805):
    ment_users = [t[:re.search('[^A-Za-z0-9_]',t).start()].lower().strip() for t in chunk if re.search('[^A-Za-z0-9_]',t) is not None]
    final_20220805.extend([[authors_20220805[n],name] for name in ment_users if len(name.strip()) > 0])
outfile = '_edgelist_20220805.csv'
save_csv(outfile,final_20220805)
top_communities_20220805 = tsm.get_top_communities(final_20220805)
nodes_data_20220805 = top_communities_20220805.node_list
calc_ei(nodes_data_20220805, final_20220805)

Data saved to file "_edgelist_20220805.csv".
Data loaded.
Non-directed network created.
Community partition complete.
Total n of communities: 155
Modularity: 0.86
Community analysis complete. The top 10 communities in this network account for 71.47 % of all nodes.
And 75.37 % of all edges.
Calculating EI indices using *weighted* edges.

Data loaded.
Data loaded.
***EI indices***

Community	EI index
0	-0.922
13	-0.977
2	-0.848
3	-0.842
33	-0.644
35	-0.227
5	-0.972
7	-0.894
9	-0.732
94	-0.654
Mean EI:	 -0.771

Community	Size	Internal	Received	Sent	Received - Sent
0	497	0.961	0.037	0.002	0.035

3-r	0.002
33-r	0.002
7-r	0.002
94-s	0.002
94-r	0.025
35-r	0.002
2-r	0.004

Community	Size	Internal	Received	Sent	Received - Sent
5	497	0.986	0.014	0.0	0.014

9-r	0.014

Community	Size	Internal	Received	Sent	Received - Sent
3	152	0.921	0.073	0.006	0.067

0-s	0.006
33-r	0.012
7-r	0.018
94-r	0.018
35-r	0.018
2-r	0.006

Community	Size	Internal	Received	Sent	Received - Sent
33	124	0.822	0.046	0.132	-0.0

In [38]:
#20220806

f = open('data/processed/tweets_20220806_notverified_freelon.csv','r',encoding = 'utf-8',errors = 'replace')
csvreader_20220806 = csv.reader(f)
authors_20220806 = []
tweets_20220806 = []
for row in csvreader_20220806:
    authors_20220806.append(re.sub('[^A-Za-z0-9_]','',str(row[1]).lower().strip()))
    tweets_20220806.append(' ' + row[2].lower() + ' ')

final_20220806 = []
tweets_20220806 = [t.split('@') for t in tweets_20220806] #splits each tweet along @s
for n,chunk in enumerate(tweets_20220806):
    ment_users = [t[:re.search('[^A-Za-z0-9_]',t).start()].lower().strip() for t in chunk if re.search('[^A-Za-z0-9_]',t) is not None]
    final_20220806.extend([[authors_20220806[n],name] for name in ment_users if len(name.strip()) > 0])
outfile = '_edgelist_20220806.csv'
save_csv(outfile,final_20220806)
top_communities_20220806 = tsm.get_top_communities(final_20220806)
nodes_data_20220806 = top_communities_20220806.node_list
calc_ei(nodes_data_20220806, final_20220806)

Data saved to file "_edgelist_20220806.csv".
Data loaded.
Non-directed network created.
Community partition complete.
Total n of communities: 157
Modularity: 0.89
Community analysis complete. The top 10 communities in this network account for 62.68 % of all nodes.
And 65.81 % of all edges.
Calculating EI indices using *weighted* edges.

Data loaded.
Data loaded.
***EI indices***

Community	EI index
1	-0.903
11	-0.964
17	-1.0
19	-0.831
2	-0.865
20	-0.792
30	-0.825
4	-0.834
5	-0.982
63	-0.97
Mean EI:	 -0.897

Community	Size	Internal	Received	Sent	Received - Sent
2	403	0.933	0.067	0.0	0.067

19-r	0.03
4-r	0.023
30-r	0.005
1-r	0.007
20-r	0.002

Community	Size	Internal	Received	Sent	Received - Sent
19	161	0.915	0.011	0.073	-0.062

2-s	0.073
4-r	0.011

Community	Size	Internal	Received	Sent	Received - Sent
11	160	0.982	0.018	0.0	0.018

5-r	0.006
63-r	0.012

Community	Size	Internal	Received	Sent	Received - Sent
4	127	0.917	0.0	0.083	-0.083

2-s	0.069
19-s	0.014

Community	Size	Internal	Receive

In [39]:
#20220807

f = open('data/processed/tweets_20220807_notverified_freelon.csv','r',encoding = 'utf-8',errors = 'replace')
csvreader_20220807 = csv.reader(f)
authors_20220807 = []
tweets_20220807 = []
for row in csvreader_20220807:
    authors_20220807.append(re.sub('[^A-Za-z0-9_]','',str(row[1]).lower().strip()))
    tweets_20220807.append(' ' + row[2].lower() + ' ')

final_20220807 = []
tweets_20220807 = [t.split('@') for t in tweets_20220807] #splits each tweet along @s
for n,chunk in enumerate(tweets_20220807):
    ment_users = [t[:re.search('[^A-Za-z0-9_]',t).start()].lower().strip() for t in chunk if re.search('[^A-Za-z0-9_]',t) is not None]
    final_20220807.extend([[authors_20220807[n],name] for name in ment_users if len(name.strip()) > 0])
outfile = '_edgelist_20220807.csv'
save_csv(outfile,final_20220807)
top_communities_20220807 = tsm.get_top_communities(final_20220807)
nodes_data_20220807 = top_communities_20220807.node_list
calc_ei(nodes_data_20220807, final_20220807)

Data saved to file "_edgelist_20220807.csv".
Data loaded.
Non-directed network created.
Community partition complete.


ValueError: A graph without link has an undefined modularity